In [1]:
import pandas as pd
import numpy as np
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
df = pd.read_csv("spam.csv", encoding="latin")

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df = df[["v1","v2"]]

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df["v1"].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [7]:
# get X as feature and Y as target 

In [8]:
lab_map = {"ham":0, "spam":1}

In [9]:
Y = df["v1"].map(lab_map).values

In [10]:
Y

array([0, 0, 1, ..., 0, 0, 0])

In [11]:
X = df["v2"].values

In [12]:
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [13]:
# convert to sequences
# pad sequence
# train

In [14]:
max_words = 100
mytokenizer = Tokenizer(nb_words=max_words,lower=True, split=" ")

/Users/tanuj.jain/anaconda3/envs/packt_env/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [15]:
mytokenizer.fit_on_texts(X)

In [17]:
text_tokenized = mytokenizer.texts_to_sequences(X)

In [18]:
text_tokenized

[[50, 64, 8, 89, 67, 58],
 [46, 6],
 [47, 8, 19, 4, 2, 71, 2, 2, 73],
 [6, 23, 6, 57],
 [1, 98, 69, 2, 69],
 [67, 21, 7, 38, 87, 55, 3, 44, 12, 14, 85, 46, 2, 68, 2],
 [11, 9, 25, 55, 2, 36, 10, 10, 55],
 [72, 13, 72, 13, 12, 51, 2, 13],
 [72, 4, 3, 17, 2, 2, 16, 64],
 [13, 96, 26, 6, 81, 2, 2, 5, 36, 12, 47, 16, 5, 96, 47, 18],
 [30, 32, 77, 7, 1, 98, 70, 2, 80, 40, 93, 88],
 [2, 48, 2, 73, 7, 68, 2, 65, 92, 42],
 [3, 17, 4, 47, 8, 91, 73, 5, 2, 38],
 [12, 5, 2, 3, 12, 40, 1, 1, 97, 13, 12, 7, 33, 11, 3, 17, 7, 4, 29, 51],
 [1, 17, 4, 18, 36, 33],
 [2, 13, 5, 8, 5, 73, 26, 89],
 [93, 30],
 [6, 49, 19, 1, 69, 1],
 [34, 5, 6, 5, 61],
 [94, 5, 73, 35, 2, 2],
 [9, 20, 49, 3],
 [75, 2, 12, 19, 64],
 [23, 57, 45, 9, 90],
 [1, 11, 57, 1, 50, 82, 82, 6, 35],
 [38, 1, 28, 44, 36, 3],
 [37, 2, 4, 30, 25, 40, 9, 69, 30, 45, 1],
 [13, 23],
 [3, 5, 22, 3, 3, 4, 22, 3, 13, 27, 3, 11, 66],
 [30, 87, 5, 21, 78, 3, 54, 34],
 [1, 20, 53, 14, 55],
 [85, 25, 51, 20, 3, 25, 80, 10, 26, 20, 70, 2, 36],
 [6

In [20]:
max_len = 50
sequences = sequence.pad_sequences(text_tokenized,maxlen=max_len)

In [21]:
sequences

array([[ 0,  0,  0, ..., 89, 67, 58],
       [ 0,  0,  0, ...,  0, 46,  6],
       [ 0,  0,  0, ...,  2,  2, 73],
       ...,
       [ 0,  0,  0, ..., 12, 20, 23],
       [ 0,  0,  0, ...,  2, 12, 47],
       [ 0,  0,  0, ..., 61,  2, 61]], dtype=int32)

In [22]:
# Build model

In [23]:
model = Sequential()
model.add(Embedding(max_words, 20, input_length=max_len))
model.add(LSTM(64))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
model.fit(sequences,Y,batch_size=128,epochs=10,
          validation_split=0.2)

Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 2s 560us/step - loss: 0.5013 - acc: 0.8503 - val_loss: 0.3872 - val_acc: 0.8700
Epoch 2/10
4457/4457 [==============================] - 2s 362us/step - loss: 0.3598 - acc: 0.8649 - val_loss: 0.3011 - val_acc: 0.8700
Epoch 3/10
4457/4457 [==============================] - 2s 356us/step - loss: 0.2269 - acc: 0.9100 - val_loss: 0.1555 - val_acc: 0.9525
Epoch 4/10
4457/4457 [==============================] - 2s 354us/step - loss: 0.1369 - acc: 0.9560 - val_loss: 0.1162 - val_acc: 0.9570
Epoch 5/10
4457/4457 [==============================] - 2s 353us/step - loss: 0.1140 - acc: 0.9614 - val_loss: 0.1067 - val_acc: 0.9605
Epoch 6/10
4457/4457 [==============================] - 2s 356us/step - loss: 0.1062 - acc: 0.9628 - val_loss: 0.1026 - val_acc: 0.9587
Epoch 7/10
4457/4457 [==============================] - 2s 384us/step - loss: 0.0995 - acc: 0.9663 - val_loss: 0.0984 - val_acc: 0.9614


In [25]:
# Predict on new test data

In [26]:
inp_test_seq = "WINNER! U win a 500 prize reward & free entry to FA cup final tickets! Text FA to 34212 to receive award"

In [27]:
test_sequences = mytokenizer.texts_to_sequences(np.array([inp_test_seq]))
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [28]:
model.predict(test_sequences_matrix)

array([[0.97663873]], dtype=float32)